In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.5'
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,535 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,000 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/ma

In [2]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [8]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [6]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
avg_price_4_bedrooms = spark.sql("""
  SELECT YEAR(date) as year_sold, ROUND(AVG(price), 2) as avg_price
  FROM home_sales
  WHERE bedrooms = 4
  GROUP BY year_sold
  ORDER BY year_sold
""")
avg_price_4_bedrooms.show()

+---------+---------+
|year_sold|avg_price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [9]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_price_3bed_3bath = spark.sql("""
  SELECT date_built, ROUND(AVG(price), 2) as avg_price
  FROM home_sales
  WHERE bedrooms = 4 AND bathrooms = 3
  GROUP BY date_built
  ORDER BY date_built
""")
avg_price_3bed_3bath.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|287529.97|
|      2011|294656.14|
|      2012|289533.13|
|      2013|291870.89|
|      2014|289935.92|
|      2015| 299517.5|
|      2016|289090.11|
|      2017|289100.17|
+----------+---------+



In [10]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg_price_filtered = spark.sql("""
  SELECT date_built, ROUND(AVG(price), 2) as avg_price
  FROM home_sales
  WHERE bedrooms = 4 AND bathrooms = 3 AND floors = 2 AND sqft_living >=2000
  GROUP BY date_built
  ORDER BY date_built
""")
avg_price_filtered.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|290237.14|
|      2011|293021.78|
|      2012|290024.97|
|      2013|293916.19|
|      2014|306202.48|
|      2015|305769.02|
|      2016|298295.66|
|      2017|289215.12|
+----------+---------+



In [11]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
start_time = time.time()
avg_price_by_view = spark.sql("""
  SELECT view, ROUND(AVG(price), 2) as avg_price
  FROM home_sales
  GROUP BY view
  HAVING avg_price >= 350000
  ORDER BY avg_price DESC
""")
avg_price_by_view.show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

--- 1.0294675827026367 seconds ---


In [12]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [13]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [14]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.
start_time = time.time()
avg_price_by_view_cached = spark.sql("""
  SELECT view, ROUND(AVG(price), 2) as avg_price
  FROM home_sales
  GROUP BY view
  HAVING avg_price >= 350000
  ORDER BY avg_price DESC
""")
avg_price_by_view_cached.show()
print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

--- 0.48294639587402344 seconds ---


In [15]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet("home_sales_partitioned.parquet", mode = "overwrite")

In [17]:
# 11. Read the parquet formatted data.
df_parquet = spark.read.parquet("home_sales_partitioned.parquet")

In [18]:
# 12. Create a temporary table for the parquet data.
df_parquet.createOrReplaceTempView("home_sales_parquet")

In [19]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.
start_time = time.time()
avg_price_by_view_parquet = spark.sql("""
  SELECT view, ROUND(AVG(price), 2) as avg_price
  FROM home_sales_parquet
  GROUP BY view
  HAVING avg_price >= 350000
  ORDER BY avg_price DESC
""")
avg_price_by_view_parquet.show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

--- 1.0900731086730957 seconds ---


In [20]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [22]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False